In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import os

# Path to dataset in your Drive
data_path = "/content/drive/MyDrive/farm_insects"

# Image generator with validation split and data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'  # Fill missing pixels after transformation
)

# Load training and validation data with augmentation
train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(256, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# EarlyStopping to stop training if the validation loss doesn't improve
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Learning rate scheduler to reduce the learning rate after a certain number of epochs
def lr_schedule(epoch, lr):
    if epoch > 15:  # Decrease learning rate after 15 epochs
        return lr * 0.5
    return lr

# Train the model with early stopping and learning rate scheduler
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping, LearningRateScheduler(lr_schedule)]
)

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")


Found 1348 images belonging to 15 classes.
Found 231 images belonging to 15 classes.
Epoch 1/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.0513 - loss: 2.7280 - val_accuracy: 0.0779 - val_loss: 2.7047 - learning_rate: 0.0010
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.0754 - loss: 2.7041 - val_accuracy: 0.0606 - val_loss: 2.6963 - learning_rate: 0.0010
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.0940 - loss: 2.6827 - val_accuracy: 0.0779 - val_loss: 2.7404 - learning_rate: 0.0010
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.1001 - loss: 2.6511 - val_accuracy: 0.0952 - val_loss: 2.6050 - learning_rate: 0.0010
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.1254 - loss: 2.5587 - val_accuracy: 0.1472 - val_loss: 2.5725 - learning_rate: 0.0010
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.1189 - loss: 2.5429 - val_accuracy: 0.1299 - val_loss: 2.5820 - learning_rate: 0.0010
Epoch 7/50
43/4